In [ ]:
path="/kaggle/input/kidneykits19/PNG_Slices_Segmented/PNG_Slices_Segmented/"
#path="../input/kidney/PNG_Slices_Segmented/PNG_Slices_Segmented/"

In [ ]:
import os
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input,Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision,Accuracy

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
H=256
W=256
dim=(H,W)

In [ ]:
def load_data(file_path):
    #Loading the image files
    images = sorted(glob(f"{file_path}/*/Images/*.png"))
    segmentations=sorted(glob(f"{file_path}/*/Segmentation/*.png"))
    print(len(images),len(segmentations))


  # #Spliting the data into training and testing using builtin libraries
    split=0.3
    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(segmentations, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)
 


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x=cv2.resize(x, dim)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x=cv2.resize(x, dim)
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


In [ ]:
(train_x, train_y), (valid_x, valid_y) = load_data(path)
print(len(train_x),len(train_y))
print(len(valid_x),len(valid_y))
train_x, train_y = shuffling(train_x, train_y)



45079 45079
31556 31556
13523 13523


In [ ]:
batch_size = 32
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [ ]:
#Metrices
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2.0 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
#Defining the structure of the model

#This convolution part will also be included in encoder and decoder part of the network
def Conv_Block(input,num_of_filter):
    x=Conv2D(num_of_filter,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation('ReLU')(x)

    x=Conv2D(num_of_filter,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation('ReLU')(x)
    return x


def Encoder(input,num_of_filter):
    x = Conv_Block(input, num_of_filter) #Skip connection
    p = MaxPool2D((2, 2))(x)  #Feature selection
    x = Dropout(0.05)(x)
    return x,p

def Decoder(input, skip_features, num_of_filter):
    x = Conv2DTranspose(num_of_filter, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = Dropout(0.05)(x)
    x = Conv_Block(x, num_of_filter)
    return x



def UNET_Build(input_shape):
    inputs=Input(input_shape)

  #as you go down in the encoder,resolution decreases , number of filters doubles
    s1,p1=Encoder(inputs,32)
    s2,p2=Encoder(p1,64)
    s3,p3=Encoder(p2,128)
    s4,p4=Encoder(p3,256)

    b1=Conv_Block(p4,512)

    d1 = Decoder(b1, s4, 256)
    d2 = Decoder(d1, s3, 128)
    d3 = Decoder(d2, s2, 64)
    d4 = Decoder(d3, s1, 32)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)  #our data is in grayscale, gives 0,1 output

    model = Model(inputs, outputs, name="U-Net")
    return model  

In [ ]:
input_shape = (256, 256, 3)
model = UNET_Build(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
lr = 0.01
num_epochs = 15

In [ ]:
model = UNET_Build((H, W, 3))
metrics = [dice_coef, iou, Recall(),Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

In [ ]:
model_weights = "/kaggle/working/model_Droput_0.05"
csv_weights = "/kaggle/working/data_model_dropout(0.05).csv"

callbacks = [
    ModelCheckpoint( model_weights,verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.00001, patience=2, min_lr=1e-7, verbose=1),
    TensorBoard(),
    CSVLogger(csv_weights),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=False),
    ]

In [ ]:
#with tf.device('/GPU:0'):
tf.debugging.set_log_device_placement(True)
with tf.device('/GPU:0'):
    print("Exection starting")
    model.fit(train_dataset,epochs=num_epochs,validation_data=valid_dataset,callbacks=callbacks,shuffle=False )

Exection starting
Epoch 1/15
987/987 [==============================] - 572s 569ms/step - loss: 0.2623 - dice_coef: 0.7380 - iou: 0.6074 - recall: 0.7255 - precision: 0.4833 - val_loss: 0.3668 - val_dice_coef: 0.6329 - val_iou: 0.4697 - val_recall: 0.4938 - val_precision: 0.8923

Epoch 00001: val_loss improved from inf to 0.36676, saving model to /kaggle/working/model_Droput_0.05
Epoch 2/15
987/987 [==============================] - 370s 374ms/step - loss: 0.0953 - dice_coef: 0.9047 - iou: 0.8294 - recall: 0.8819 - precision: 0.9307 - val_loss: 0.2138 - val_dice_coef: 0.7860 - val_iou: 0.6514 - val_recall: 0.7528 - val_precision: 0.8288

Epoch 00002: val_loss improved from 0.36676 to 0.21379, saving model to /kaggle/working/model_Droput_0.05
Epoch 3/15
987/987 [==============================] - 365s 370ms/step - loss: 0.0554 - dice_coef: 0.9446 - iou: 0.8964 - recall: 0.9348 - precision: 0.9558 - val_loss: 0.2679 - val_dice_coef: 0.7317 - val_iou: 0.5821 - val_recall: 0.7429 - val_prec

In [ ]:
model.save("UNET_Dropout(0.05).h5")